In [2]:
import os
import pathlib
import sys
import matplotlib.pyplot as plt
from scipy import linalg


sys.path.append(os.path.join(pathlib.Path(os.getcwd()).parent))
import pandas as pd
from datetime import datetime

from utils.bq_conn import bq_conn_file
# %%
bq_client = bq_conn_file(project_id="dsa-miad-365616")

ModuleNotFoundError: No module named 'utils.bq_conn'

In [ ]:
query = '''
SELECT
    *
FROM `Proyecto.DatosSecopII`
ORDER BY estado_contrato DESC
'''
secop_II = bq_client.execute_query(query=query)
secop_II.head(3)

In [ ]:
query = '''
SELECT
    *
FROM `Proyecto.DatosSecopII`
ORDER BY estado_contrato DESC
'''
secop_I = bq_client.execute_query(query=query)
secop_I.head(3)

In [ ]:
secop_I.columns

In [ ]:
base=secop_I[["estado_contrato", "tipo_de_contrato", "modalidad_de_contratacion", "justificacion_modalidad_de", "destino_gasto", "nombre_representante_legal", "identificaci_n_representante_legal", "objeto_del_contrato", "prorrogado","dias_adicionados"]]

In [ ]:
base.info()

In [ ]:
n = base.isna()== True 
base = base.fillna(0)
base.info()

data_numeric = pd.get_dummies(base)

# calcular la matriz de correlación de Pearson
correlation_matrix = data_numeric.corr(method='pearson')

# imprimir matriz de correlación
print(correlation_matrix)

In [ ]:
# Codificación de las variables categoricas
base['estado_contrato'] = pd.factorize(base.estado_contrato)[0]
base['tipo_de_contrato'] = pd.factorize(base.tipo_de_contrato)[0]
base['modalidad_de_contratacion'] = pd.factorize(base.modalidad_de_contratacion)[0]
base['justificacion_modalidad_de'] = pd.factorize(base.justificacion_modalidad_de)[0]
base['destino_gasto'] = pd.factorize(base.destino_gasto)[0]
base['nombre_representante_legal'] = pd.factorize(base.nombre_representante_legal)[0]
base['objeto_del_contrato'] = pd.factorize(base.objeto_del_contrato)[0]
base['identificaci_n_representante_legal'] = pd.factorize(base.identificaci_n_representante_legal)[0]


In [ ]:
base

In [ ]:
import seaborn as sns
import numpy as np
sns.heatmap(base.corr(), vmin=-1, vmax=1, annot=True)


En el anterior mapa de calor no encontramos, que todas las variables son independientes, con excepción de el nombre y la identificación del contratista, aunque existe cierta relación objetiva (>85%) entr el objeto del contrato y el estaod del mismo. Para nuestra variable de interés que es la catidad de dîas adicionados no parece haber una verdadera relación entre esta y las demás variables. Por lo que sin ser concluyente, puede decirse que no parecería haber una dependencia entre las prórrogas y la mayoría de los atributos de los contratos.

No obstante la factorización, no siempre es la mejor opción para verificar correlación entre variable categóricas. Por lo cual deben evaluarse nuevas formas de evaluación de la dependencia o no entre las variables del contrato y su prorrogación. 


In [ ]:
S = base.cov()
eig_val, eig_vec = np.linalg.eig(S)
eig_val

eig_val_diag = np.diag(eig_val)
eig_val

In [ ]:
vector = eig_val/sum(eig_val)
vector

Efectuando la selección de atributos principales encontramos que los eigenvalores diagonales de la matriz de covarianzas, muestra importancia superior en el atriubuto "destino del gasto" como componente principal de la matriz para explicar los otros atributos. 

In [ ]:
eigenv=pd.DataFrame(eig_vec.dot(eig_val_diag).dot(eig_vec.T), columns= S.columns, index= S.index)

In [ ]:
sns.pairplot(eigenv)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from pandas_profiling import ProfileReport
from sklearn.decomposition import PCA

In [ ]:
X = base[['estado_contrato','tipo_de_contrato', 'modalidad_de_contratacion','justificacion_modalidad_de','destino_gasto', 'nombre_representante_legal', 'identificaci_n_representante_legal', 'objeto_del_contrato','prorrogado']]
y = (base.dias_adicionados)

In [ ]:
pca = PCA(n_components=4)
>>> pca.fit(X)

In [ ]:
fig, ax = plt.subplots()    
ax.plot(pca.explained_variance_)
ax.xaxis.set_major_locator
ax.set_xlabel('Principal Component')
ax.set_ylabel('Eigenvalue')
plt.xticks(np.arange(9), np.arange(1, 10))
ax.title.set_text('Componentes principales')
plt.show()


In [ ]:
profile = ProfileReport(base, title = "Profiling Report", explorative = True)
profile.to_widgets()